In [1]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import rasters as rt
from rasters import MultiPoint
from PTJPL import PTJPL, process_PTJPL_table, load_ECOv002_static_tower_PTJPL_inputs
from PTJPL import load_Topt, load_fAPARmax
from ECOv002_calval_tables import load_combined_eco_flux_ec_filtered, load_metadata_ebc_filt, load_calval_table

In [2]:
repo_root = os.path.dirname(os.getcwd())
package_dir = os.path.join(repo_root, 'PTJPL')
generated_input_table_filename = os.path.join(package_dir, "ECOv002-cal-val-PT-JPL-inputs.csv")
generated_output_table_filename = os.path.join(package_dir, "ECOv002-cal-val-PT-JPL-outputs.csv")

In [3]:
model_inputs_gdf = load_calval_table()
model_inputs_gdf

,Unnamed: 0,ID,vegetation,climate,STICinst,BESSinst,MOD16inst,PTJPLSMinst,ETinst,ETinstUncertainty,...,EndDate,LE_count,closure_ratio,geometry,time_UTC,ST_K,ST_C,Ta_C,SWin_Wm2,emissivity
0,0,US-NC3,ENF,Cfa,270.345200,78.53355,392.851840,307.021970,487.383423,118.916280,...,1/1/22 05:00,9576,1.02,POINT (-76.656 35.799),2019-10-02 19:09:40,305.10,31.95,32.658920,545.51056,0.948
1,1,US-Mi3,CVM,Dfb,232.141600,229.20093,640.118470,375.089300,106.825577,167.919460,...,12/28/19 04:00,12170,0.92,POINT (-80.637 41.8222),2019-06-23 18:17:17,304.34,31.19,24.227982,848.34390,0.952
2,2,US-Mi3,CVM,Dfb,356.355740,335.23154,625.661700,284.686250,NaN,132.936340,...,12/28/19 04:00,12170,0.92,POINT (-80.637 41.8222),2019-06-27 16:35:42,304.06,30.91,26.178862,838.81160,0.972
3,3,US-Mi3,CVM,Dfb,332.938400,326.68680,624.254330,251.414490,178.827545,141.132420,...,12/28/19 04:00,12170,0.92,POINT (-80.637 41.8222),2019-06-30 15:44:10,301.80,28.65,22.527096,851.72480,0.974
4,4,US-Mi3,CVM,Dfb,286.854030,237.21654,511.082180,228.520170,154.791626,114.809410,...,12/28/19 04:00,12170,0.92,POINT (-80.637 41.8222),2019-07-01 14:53:48,303.18,30.03,23.280691,702.55160,0.960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060,1060,US-xAE,GRA,Cfa,70.923310,172.37459,81.645230,15.282976,NaN,56.385185,...,12/1/22 00:00,29615,0.60,POINT (-99.0588 35.4106),2021-12-11 16:01:12,278.78,5.63,3.815752,286.84660,0.980
1061,1061,US-xAE,GRA,Cfa,116.543190,121.81641,65.469320,22.186659,NaN,40.509410,...,12/1/22 00:00,29615,0.60,POINT (-99.0588 35.4106),2022-03-25 22:45:31,293.28,20.13,19.266186,290.87400,0.976
1062,1062,US-xAE,GRA,Cfa,129.880100,0.00000,118.777240,55.343586,NaN,52.403820,...,12/1/22 00:00,29615,0.60,POINT (-99.0588 35.4106),2022-04-12 22:53:09,301.94,28.79,32.110336,352.19530,0.976
1063,1063,US-xAE,GRA,Cfa,2.707851,140.38632,126.490524,40.434025,NaN,57.769722,...,12/1/22 00:00,29615,0.60,POINT (-99.0588 35.4106),2022-04-14 14:45:37,290.72,17.57,10.464681,420.67880,0.976


In [4]:
static_inputs_df = load_ECOv002_static_tower_PTJPL_inputs()
static_inputs_df

,ID,name,Topt_C,fAPARmax,geometry
0,US-NC3,NC_Clearcut#3,10.09,0.4659,POINT (-76.656 35.799)
1,PE-QFR,Quistococha Forest Reserve,25.06,0.5924,POINT (-73.319 -3.8344)
2,US-Mi3,LTAR UCB (Upper Chesapeake Bay) Miscanthus 3,1.39,0.4865,POINT (-80.637 41.8222)
3,US-NC4,NC_AlligatorRiver,9.74,0.5777,POINT (-75.9038 35.7879)
4,CA-DB2,Delta Burns Bog 2,3.43,0.4949,POINT (-122.9951 49.119)
...,...,...,...,...,...
116,US-xSL,"NEON North Sterling, CO (STER)",0.66,0.3046,POINT (-103.0293 40.4619)
117,US-xWD,NEON Woodworth (WOOD),0.00,0.3626,POINT (-99.2414 47.1282)
118,US-CS4,Central Sands Irrigated Agricultural Field,0.00,0.3903,POINT (-89.5475 44.1597)
119,US-xAE,NEON Klemme Range Research Station (OAES),7.99,0.3387,POINT (-99.0588 35.4106)


In [5]:
# extract array of x and array of y coordinates from data frame geometry
x_coords = model_inputs_gdf.geometry.x
y_coords = model_inputs_gdf.geometry.y
tower_points = MultiPoint(x=x_coords, y=y_coords)
tower_points

MULTIPOINT ((-76.656 35.799), (-80.637 41.8222), (-80.637 41.8222), (-80.637 41.8222), (-80.637 41.8222), (-80.637 41.8222), (-75.9038 35.7879), (-80.6313 41.7727), (-80.6313 41.7727), (-80.6313 41.7727), (-80.6313 41.7727), (-80.6313 41.7727), (-90.3004 45.9793), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-122.3303 45.7624), (-71.7181 43.9397), (-71.7181 43.9397), (-71.7181 43.9397), (-71.7181 43.9397), (-71.7181 43.9397), (-71.7181 43.9397), (-71.7181 43.9397), (-121.6078 44.3233), (-121.6078 44.3233), (-121.6078 44.3233), (-121.6078 44.3233), (-121.6078 44.3233), (-121.6078 44.3233), (-121.6078 44.3233), (-121.6078 44.3233), (-121.6078 44.3233), (-121.6078 44.3233), (-121.6078 44.3233), (-121.6

In [6]:
# merge static inputs with model inputs, ignoring duplicate columns from static_inputs_df
cols_to_use = [col for col in static_inputs_df.columns if col not in model_inputs_gdf.columns or col == 'ID']
model_inputs_gdf = model_inputs_gdf.merge(static_inputs_df[cols_to_use], on="ID", how="left")
model_inputs_gdf

,Unnamed: 0,ID,vegetation,climate,STICinst,BESSinst,MOD16inst,PTJPLSMinst,ETinst,ETinstUncertainty,...,geometry,time_UTC,ST_K,ST_C,Ta_C,SWin_Wm2,emissivity,name,Topt_C,fAPARmax
0,0,US-NC3,ENF,Cfa,270.345200,78.53355,392.851840,307.021970,487.383423,118.916280,...,POINT (-76.656 35.799),2019-10-02 19:09:40,305.10,31.95,32.658920,545.51056,0.948,NC_Clearcut#3,10.09,0.4659
1,1,US-Mi3,CVM,Dfb,232.141600,229.20093,640.118470,375.089300,106.825577,167.919460,...,POINT (-80.637 41.8222),2019-06-23 18:17:17,304.34,31.19,24.227982,848.34390,0.952,LTAR UCB (Upper Chesapeake Bay) Miscanthus 3,1.39,0.4865
2,2,US-Mi3,CVM,Dfb,356.355740,335.23154,625.661700,284.686250,NaN,132.936340,...,POINT (-80.637 41.8222),2019-06-27 16:35:42,304.06,30.91,26.178862,838.81160,0.972,LTAR UCB (Upper Chesapeake Bay) Miscanthus 3,1.39,0.4865
3,3,US-Mi3,CVM,Dfb,332.938400,326.68680,624.254330,251.414490,178.827545,141.132420,...,POINT (-80.637 41.8222),2019-06-30 15:44:10,301.80,28.65,22.527096,851.72480,0.974,LTAR UCB (Upper Chesapeake Bay) Miscanthus 3,1.39,0.4865
4,4,US-Mi3,CVM,Dfb,286.854030,237.21654,511.082180,228.520170,154.791626,114.809410,...,POINT (-80.637 41.8222),2019-07-01 14:53:48,303.18,30.03,23.280691,702.55160,0.960,LTAR UCB (Upper Chesapeake Bay) Miscanthus 3,1.39,0.4865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060,1060,US-xAE,GRA,Cfa,70.923310,172.37459,81.645230,15.282976,NaN,56.385185,...,POINT (-99.0588 35.4106),2021-12-11 16:01:12,278.78,5.63,3.815752,286.84660,0.980,NEON Klemme Range Research Station (OAES),7.99,0.3387
1061,1061,US-xAE,GRA,Cfa,116.543190,121.81641,65.469320,22.186659,NaN,40.509410,...,POINT (-99.0588 35.4106),2022-03-25 22:45:31,293.28,20.13,19.266186,290.87400,0.976,NEON Klemme Range Research Station (OAES),7.99,0.3387
1062,1062,US-xAE,GRA,Cfa,129.880100,0.00000,118.777240,55.343586,NaN,52.403820,...,POINT (-99.0588 35.4106),2022-04-12 22:53:09,301.94,28.79,32.110336,352.19530,0.976,NEON Klemme Range Research Station (OAES),7.99,0.3387
1063,1063,US-xAE,GRA,Cfa,2.707851,140.38632,126.490524,40.434025,NaN,57.769722,...,POINT (-99.0588 35.4106),2022-04-14 14:45:37,290.72,17.57,10.464681,420.67880,0.976,NEON Klemme Range Research Station (OAES),7.99,0.3387


In [7]:
model_inputs_gdf.keys()

Index(['Unnamed: 0', 'ID', 'vegetation', 'climate', 'STICinst', 'BESSinst',
       'MOD16inst', 'PTJPLSMinst', 'ETinst', 'ETinstUncertainty', 'PET', 'Rn',
       'ESI', 'RH', 'Ta', 'LST', 'SM', 'NDVI', 'NDVI-UQ', 'albedo',
       'albedo-UQ', 'LST_err', 'view_zenith', 'Rg', 'EmisWB', 'time_utc',
       'solar_time', 'solar_hour', 'local_time', 'LE', 'LE_filt', 'LEcorr25',
       'LEcorr50', 'LEcorr75', 'LEcorr_ann', 'H_filt', 'Hcorr25', 'Hcorr50',
       'Hcorr75', 'Hcorr_ann', 'NETRAD_filt', 'G_filt', 'SM_surf', 'SM_rz',
       'AirTempC', 'SW_IN', 'RH_percentage', 'ESIrn_STIC', 'ESIrn_PTJPLSM',
       'ESIrn_MOD16', 'ESIrn_BESS', 'ESIrn_Unc_ECO', 'ESIrn_LEcorr50', 'JET',
       'eco_time_utc', 'Site Name', 'Date-Time', 'Site ID', 'Name', 'Lat',
       'Long', 'Elev', 'Clim', 'Veg', 'MAT', 'MAP', 'StartDate', 'EndDate',
       'LE_count', 'closure_ratio', 'geometry', 'time_UTC', 'ST_K', 'ST_C',
       'Ta_C', 'SWin_Wm2', 'emissivity', 'name', 'Topt_C', 'fAPARmax'],
      dtype='object

In [8]:
model_inputs_gdf.albedo

0       0.215445
1       0.117238
2       0.117280
3       0.084629
4       0.120526
          ...   
1060    0.092853
1061    0.111844
1062    0.106782
1063    0.106775
1064    0.113165
Name: albedo, Length: 1065, dtype: float64

In [9]:
results = process_PTJPL_table(model_inputs_gdf)
results

,Unnamed: 0,ID,vegetation,climate,STICinst,BESSinst,MOD16inst,PTJPLSMinst,ETinst,ETinstUncertainty,...,Topt_C,fAPARmax,G_Wm2,Rn_soil_Wm2,LE_soil_Wm2,Rn_canopy_Wm2,PET_Wm2,LE_canopy_Wm2,LE_interception_Wm2,LE_Wm2
0,0,US-NC3,ENF,Cfa,270.345200,78.53355,392.851840,307.021970,487.383423,118.916280,...,10.09,0.4659,51.001527,108.025913,16.547920,285.831187,348.579227,249.872119,0.029060,266.449099
1,1,US-Mi3,CVM,Dfb,232.141600,229.20093,640.118470,375.089300,106.825577,167.919460,...,1.39,0.4865,81.309890,242.983670,41.693402,400.491330,518.484847,334.616839,0.036937,376.347178
2,2,US-Mi3,CVM,Dfb,356.355740,335.23154,625.661700,284.686250,NaN,132.936340,...,1.39,0.4865,80.764205,255.079320,89.714942,373.689380,518.782458,325.830086,0.035376,415.580404
3,3,US-Mi3,CVM,Dfb,332.938400,326.68680,624.254330,251.414490,178.827545,141.132420,...,1.39,0.4865,64.855799,203.478663,76.937311,423.480937,505.753641,335.529206,0.038103,412.504620
4,4,US-Mi3,CVM,Dfb,286.854030,237.21654,511.082180,228.520170,154.791626,114.809410,...,1.39,0.4865,58.296844,162.335322,47.807718,350.798478,413.852320,279.933774,0.031919,327.773411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060,1060,US-xAE,GRA,Cfa,70.923310,172.37459,81.645230,15.282976,NaN,56.385185,...,7.99,0.3387,4.103044,121.450797,38.910376,41.796593,92.388700,16.172143,0.002426,55.084946
1061,1061,US-xAE,GRA,Cfa,116.543190,121.81641,65.469320,22.186659,NaN,40.509410,...,7.99,0.3387,15.442521,131.899739,28.754699,34.292221,128.608037,23.325564,0.002926,52.083188
1062,1062,US-xAE,GRA,Cfa,129.880100,0.00000,118.777240,55.343586,NaN,52.403820,...,7.99,0.3387,30.976170,171.573784,4.856571,64.307806,207.238994,58.667346,0.006504,63.530420
1063,1063,US-xAE,GRA,Cfa,2.707851,140.38632,126.490524,40.434025,NaN,57.769722,...,7.99,0.3387,19.863577,180.002902,47.538057,67.875998,161.062364,43.339051,0.004795,90.881902


In [10]:
model_inputs_gdf.to_csv(generated_input_table_filename, index=False)

In [11]:
results.to_csv(generated_output_table_filename, index=False)